**Chapter 19 - Training and Deploying Models at Scale**

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/ageron/handson-ml3/blob/main/19_training_and_deploying_at_scale.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
  </td>
  <td>
    <a target="_blank" href="https://kaggle.com/kernels/welcome?src=https://github.com/ageron/handson-ml3/blob/main/19_training_and_deploying_at_scale.ipynb"><img src="https://kaggle.com/static/images/open-in-kaggle.svg" /></a>
  </td>
</table>

# Setup

This project requires Python 3.7 or above:

In [1]:
import sys

assert sys.version_info >= (3, 7)

And TensorFlow ≥ 2.8:

In [2]:
from packaging import version
import tensorflow as tf

assert version.parse(tf.__version__) >= version.parse("2.8.0")

2023-05-21 22:08:11.741103: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-21 22:08:17.244152: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-05-21 22:08:17.244191: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-05-21 22:08:39.610439: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

If running on Colab or Kaggle, you need to install the Google AI Platform client library, which will be used later in this notebook. You can ignore the warnings about version incompatibilities.

* **Warning**: On Colab, you must restart the Runtime after the installation, and continue with the next cells.

In [3]:
import sys
if "google.colab" in sys.modules or "kaggle_secrets" in sys.modules:
    %pip install -q -U google-cloud-aiplatform

This chapter discusses how to run or train a model on one or more GPUs, so let's make sure there's at least one, or else issue a warning:

In [4]:
if not tf.config.list_physical_devices('GPU'):
    print("No GPU was detected. Neural nets can be very slow without a GPU.")
    if "google.colab" in sys.modules:
        print("Go to Runtime > Change runtime and select a GPU hardware "
              "accelerator.")
    if "kaggle_secrets" in sys.modules:
        print("Go to Settings > Accelerator and select GPU.")

No GPU was detected. Neural nets can be very slow without a GPU.


2023-05-21 22:09:09.285944: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-05-21 22:09:09.295793: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-05-21 22:09:09.296052: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (avtr): /proc/driver/nvidia/version does not exist


# Serving a TensorFlow Model

Let's start by deploying a model using TF Serving, then we'll deploy to Google Vertex AI.

## Using TensorFlow Serving

The first thing we need to do is to build and train a model, and export it to the SavedModel format.

### Exporting SavedModels

Let's load the MNIST dataset, scale it, and split it.

In [5]:
from pathlib import Path
import tensorflow as tf

# extra code – load and split the MNIST dataset
mnist = tf.keras.datasets.mnist.load_data()
(X_train_full, y_train_full), (X_test, y_test) = mnist
X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

# extra code – build & train an MNIST model (also handles image preprocessing)
tf.random.set_seed(42)
tf.keras.backend.clear_session()
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=[28, 28], dtype=tf.uint8),
    tf.keras.layers.Rescaling(scale=1 / 255),
    tf.keras.layers.Dense(100, activation="relu"),
    tf.keras.layers.Dense(10, activation="softmax")
])
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=tf.keras.optimizers.SGD(learning_rate=1e-2),
              metrics=["accuracy"])
model.fit(X_train, y_train, epochs=10, validation_data=(X_valid, y_valid))

model_name = "my_mnist_model"
model_version = "0001"
model_path = Path(model_name) / model_version
model.save(model_path, save_format="tf")

11490434/11490434 [==============================] - 7s 1us/step


2023-05-21 22:11:56.294548: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Epoch 1/10
1719/1719 [==============================] - 18s 8ms/step - loss: 0.6984 - accuracy: 0.8221 - val_loss: 0.3734 - val_accuracy: 0.8996
Epoch 2/10
1719/1719 [==============================] - 11s 6ms/step - loss: 0.3513 - accuracy: 0.9021 - val_loss: 0.3010 - val_accuracy: 0.9140
Epoch 3/10
1719/1719 [==============================] - 11s 6ms/step - loss: 0.3000 - accuracy: 0.9149 - val_loss: 0.2653 - val_accuracy: 0.9282
Epoch 4/10
1719/1719 [==============================] - 8s 5ms/step - loss: 0.2689 - accuracy: 0.9241 - val_loss: 0.2426 - val_accuracy: 0.9338
Epoch 5/10
1719/1719 [==============================] - 7s 4ms/step - loss: 0.2457 - accuracy: 0.9307 - val_loss: 0.2238 - val_accuracy: 0.9374
Epoch 6/10
1719/1719 [==============================] - 6s 4ms/step - loss: 0.2272 - accuracy: 0.9359 - val_loss: 0.2102 - val_accuracy: 0.9422
Epoch 7/10
1719/1719 [==============================] - 7s 4ms/step - loss: 0.2117 - accuracy: 0.9405 - val_loss: 0.1953 - val_accura

INFO:tensorflow:Assets written to: my_mnist_model/0001/assets


INFO:tensorflow:Assets written to: my_mnist_model/0001/assets


Let's take a look at the file tree (we've discussed what each of these file is used for in chapter 10):

In [6]:
sorted([str(path) for path in model_path.parent.glob("**/*")])  # extra code

['my_mnist_model/0001',
 'my_mnist_model/0001/assets',
 'my_mnist_model/0001/fingerprint.pb',
 'my_mnist_model/0001/keras_metadata.pb',
 'my_mnist_model/0001/saved_model.pb',
 'my_mnist_model/0001/variables',
 'my_mnist_model/0001/variables/variables.data-00000-of-00001',
 'my_mnist_model/0001/variables/variables.index']

Let's inspect the SavedModel:

In [7]:
!saved_model_cli show --dir '{model_path}'

2023-05-21 22:13:31.910965: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-21 22:13:32.133569: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-05-21 22:13:32.133603: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-05-21 22:13:34.174534: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

In [8]:
!saved_model_cli show --dir '{model_path}' --tag_set serve

2023-05-21 22:13:42.732275: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-21 22:13:42.949183: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-05-21 22:13:42.949217: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-05-21 22:13:44.392802: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

In [9]:
!saved_model_cli show --dir '{model_path}' --tag_set serve \
                      --signature_def serving_default

2023-05-21 22:13:47.818497: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-21 22:13:48.114632: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-05-21 22:13:48.114666: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-05-21 22:13:49.377121: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

For even more details, you can run the following command:

```ipython
!saved_model_cli show --dir '{model_path}' --all
```

###  Installing and Starting TensorFlow Serving

If you are running this notebook in Colab or Kaggle, TensorFlow Server needs to be installed:

In [10]:
if "google.colab" in sys.modules or "kaggle_secrets" in sys.modules:
    url = "https://storage.googleapis.com/tensorflow-serving-apt"
    src = "stable tensorflow-model-server tensorflow-model-server-universal"
    !echo 'deb {url} {src}' > /etc/apt/sources.list.d/tensorflow-serving.list
    !curl '{url}/tensorflow-serving.release.pub.gpg' | apt-key add -
    !apt update -q && apt-get install -y tensorflow-model-server
    %pip install -q -U tensorflow-serving-api

If `tensorflow_model_server` is installed (e.g., if you are running this notebook in Colab), then the following 2 cells will start the server. If your OS is Windows, you may need to run the `tensorflow_model_server` command in a terminal, and replace `${MODEL_DIR}` with the full path to the `my_mnist_model` directory.

In [11]:
import os

os.environ["MODEL_DIR"] = str(model_path.parent.absolute())

In [12]:
%%bash --bg
tensorflow_model_server \
    --port=8500 \
    --rest_api_port=8501 \
    --model_name=my_mnist_model \
    --model_base_path="${MODEL_DIR}" >my_server.log 2>&1

If you are running this notebook on your own machine, and you prefer to install TF Serving using Docker, first make sure [Docker](https://docs.docker.com/install/) is installed, then run the following commands in a terminal. You must replace `/path/to/my_mnist_model` with the appropriate absolute path to the `my_mnist_model` directory, but do not modify the container path `/models/my_mnist_model`.

```bash
docker pull tensorflow/serving  # downloads the latest TF Serving image

docker run -it --rm -v "/path/to/my_mnist_model:/models/my_mnist_model" \
    -p 8500:8500 -p 8501:8501 -e MODEL_NAME=my_mnist_model tensorflow/serving
```

### Querying TF Serving through the REST API

Next, let's send a REST query to TF Serving:

In [13]:
import json

X_new = X_test[:3]  # pretend we have 3 new digit images to classify
request_json = json.dumps({
    "signature_name": "serving_default",
    "instances": X_new.tolist(),
})

In [15]:
request_json[:100] + "..." + request_json[-10:]

'{"signature_name": "serving_default", "instances": [[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0..., 0, 0]]]}'

Now let's use TensorFlow Serving's REST API to make predictions:

In [ ]:
import requests

server_url = "http://localhost:8501/v1/models/my_mnist_model:predict"
response = requests.post(server_url, data=request_json)
response.raise_for_status()  # raise an exception in case of error
response = response.json()

In [ ]:
import numpy as np

y_proba = np.array(response["predictions"])
y_proba.round(2)

### Querying TF Serving through gRPC API

In [ ]:
from tensorflow_serving.apis.predict_pb2 import PredictRequest

request = PredictRequest()
request.model_spec.name = model_name
request.model_spec.signature_name = "serving_default"
input_name = model.input_names[0]  # == "flatten_input"
request.inputs[input_name].CopyFrom(tf.make_tensor_proto(X_new))

In [ ]:
import grpc
from tensorflow_serving.apis import prediction_service_pb2_grpc

channel = grpc.insecure_channel('localhost:8500')
predict_service = prediction_service_pb2_grpc.PredictionServiceStub(channel)
response = predict_service.Predict(request, timeout=10.0)

Convert the response to a tensor:

In [ ]:
output_name = model.output_names[0]
outputs_proto = response.outputs[output_name]
y_proba = tf.make_ndarray(outputs_proto)

In [ ]:
y_proba.round(2)

If your client does not include the TensorFlow library, you can convert the response to a NumPy array like this:

In [ ]:
# extra code – shows how to avoid using tf.make_ndarray()
output_name = model.output_names[0]
outputs_proto = response.outputs[output_name]
shape = [dim.size for dim in outputs_proto.tensor_shape.dim]
y_proba = np.array(outputs_proto.float_val).reshape(shape)
y_proba.round(2)

### Deploying a new model version

In [21]:
# extra code – build and train a new MNIST model version
np.random.seed(42)
tf.random.set_seed(42)
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=[28, 28], dtype=tf.uint8),
    tf.keras.layers.Rescaling(scale=1 / 255),
    tf.keras.layers.Dense(50, activation="relu"),
    tf.keras.layers.Dense(50, activation="relu"),
    tf.keras.layers.Dense(10, activation="softmax")
])
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=tf.keras.optimizers.SGD(learning_rate=1e-2),
              metrics=["accuracy"])
history = model.fit(X_train, y_train, epochs=10,
                    validation_data=(X_valid, y_valid))

Epoch 1/10
1719/1719 [==============================] - 13s 6ms/step - loss: 0.7077 - accuracy: 0.8062 - val_loss: 0.3300 - val_accuracy: 0.9038
Epoch 2/10
1719/1719 [==============================] - 13s 7ms/step - loss: 0.3133 - accuracy: 0.9096 - val_loss: 0.2640 - val_accuracy: 0.9230
Epoch 3/10
1719/1719 [==============================] - 11s 7ms/step - loss: 0.2640 - accuracy: 0.9239 - val_loss: 0.2268 - val_accuracy: 0.9330
Epoch 4/10
1719/1719 [==============================] - 8s 4ms/step - loss: 0.2314 - accuracy: 0.9331 - val_loss: 0.2024 - val_accuracy: 0.9422
Epoch 5/10
1719/1719 [==============================] - 7s 4ms/step - loss: 0.2068 - accuracy: 0.9408 - val_loss: 0.1851 - val_accuracy: 0.9454
Epoch 6/10
1719/1719 [==============================] - 8s 5ms/step - loss: 0.1873 - accuracy: 0.9468 - val_loss: 0.1719 - val_accuracy: 0.9520
Epoch 7/10
1719/1719 [==============================] - 9s 5ms/step - loss: 0.1722 - accuracy: 0.9515 - val_loss: 0.1593 - val_accura

In [22]:
model_version = "0002"
model_path = Path(model_name) / model_version
model.save(model_path, save_format="tf")

INFO:tensorflow:Assets written to: my_mnist_model/0002/assets


INFO:tensorflow:Assets written to: my_mnist_model/0002/assets


Let's take a look at the file tree again:

In [23]:
sorted([str(path) for path in model_path.parent.glob("**/*")])  # extra code

['my_mnist_model/0001',
 'my_mnist_model/0001/assets',
 'my_mnist_model/0001/fingerprint.pb',
 'my_mnist_model/0001/keras_metadata.pb',
 'my_mnist_model/0001/saved_model.pb',
 'my_mnist_model/0001/variables',
 'my_mnist_model/0001/variables/variables.data-00000-of-00001',
 'my_mnist_model/0001/variables/variables.index',
 'my_mnist_model/0002',
 'my_mnist_model/0002/assets',
 'my_mnist_model/0002/fingerprint.pb',
 'my_mnist_model/0002/keras_metadata.pb',
 'my_mnist_model/0002/saved_model.pb',
 'my_mnist_model/0002/variables',
 'my_mnist_model/0002/variables/variables.data-00000-of-00001',
 'my_mnist_model/0002/variables/variables.index']

**Warning**: You may need to wait a minute before the new model is loaded by TensorFlow Serving.

In [ ]:
import requests

server_url = "http://localhost:8501/v1/models/my_mnist_model:predict"
            
response = requests.post(server_url, data=request_json)
response.raise_for_status()
response = response.json()

In [ ]:
response.keys()

In [ ]:
y_proba = np.array(response["predictions"])
y_proba.round(2)

## Creating a Prediction Service on Vertex AI

Follow the instructions in the book to create a Google Cloud Platform account and activate the Vertex AI and Cloud Storage APIs. Then, if you're running this notebook in Colab, you can run the following cell to authenticate using the same Google account as you used with Google Cloud Platform, and authorize this Colab to access your data.

**WARNING: only do this if you trust this notebook!**
* Be extra careful if this is not the official notebook from https://github.com/ageron/handson-ml3: the Colab URL should start with https://colab.research.google.com/github/ageron/handson-ml3. Or else, the code could do whatever it wants with your data.

If you are not running this notebook in Colab, you must follow the instructions in the book to create a service account and generate a key for it, download it to this notebook's directory, and name it `my_service_account_key.json` (or make sure the `GOOGLE_APPLICATION_CREDENTIALS` environment variable points to your key).

In [ ]:
project_id = "my_project"  ##### CHANGE THIS TO YOUR PROJECT ID #####

if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user()
elif "kaggle_secrets" in sys.modules:
    from kaggle_secrets import UserSecretsClient
    UserSecretsClient().set_gcloud_credentials(project=project_id)
else:
    os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "my_service_account_key.json"

In [ ]:
from google.cloud import storage

bucket_name = "my_bucket"  ##### CHANGE THIS TO A UNIQUE BUCKET NAME #####
location = "us-central1"

storage_client = storage.Client(project=project_id)
bucket = storage_client.create_bucket(bucket_name, location=location)
#bucket = storage_client.bucket(bucket_name)  # to reuse a bucket instead

In [ ]:
def upload_directory(bucket, dirpath):
    dirpath = Path(dirpath)
    for filepath in dirpath.glob("**/*"):
        if filepath.is_file():
            blob = bucket.blob(filepath.relative_to(dirpath.parent).as_posix())
            blob.upload_from_filename(filepath)

upload_directory(bucket, "my_mnist_model")

In [ ]:
# extra code – a much faster multithreaded implementation of upload_directory()
#              which also accepts a prefix for the target path, and prints stuff

from concurrent import futures

def upload_file(bucket, filepath, blob_path):
    blob = bucket.blob(blob_path)
    blob.upload_from_filename(filepath)

def upload_directory(bucket, dirpath, prefix=None, max_workers=50):
    dirpath = Path(dirpath)
    prefix = prefix or dirpath.name
    with futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
        future_to_filepath = {
            executor.submit(
                upload_file,
                bucket, filepath,
                f"{prefix}/{filepath.relative_to(dirpath).as_posix()}"
            ): filepath
            for filepath in sorted(dirpath.glob("**/*"))
            if filepath.is_file()
        }
        for future in futures.as_completed(future_to_filepath):
            filepath = future_to_filepath[future]
            try:
                result = future.result()
            except Exception as ex:
                print(f"Error uploading {filepath!s:60}: {ex}")  # f!s is str(f)
            else:
                print(f"Uploaded {filepath!s:60}", end="\r")

    print(f"Uploaded {dirpath!s:60}")

Alternatively, if you installed Google Cloud CLI (it's preinstalled on Colab), then you can use the following `gsutil` command:

In [25]:
#!gsutil -m cp -r my_mnist_model gs://{bucket_name}/

In [ ]:
from google.cloud import aiplatform

server_image = "gcr.io/cloud-aiplatform/prediction/tf2-gpu.2-8:latest"

aiplatform.init(project=project_id, location=location)
mnist_model = aiplatform.Model.upload(
    display_name="mnist",
    artifact_uri=f"gs://{bucket_name}/my_mnist_model/0001",
    serving_container_image_uri=server_image,
)

**Warning**: this cell may take several minutes to run, as it waits for Vertex AI to provision the compute nodes:

In [ ]:
endpoint = aiplatform.Endpoint.create(display_name="mnist-endpoint")

endpoint.deploy(
    mnist_model,
    min_replica_count=1,
    max_replica_count=5,
    machine_type="n1-standard-4",
    accelerator_type="NVIDIA_TESLA_K80",
    accelerator_count=1
)

In [ ]:
response = endpoint.predict(instances=X_new.tolist())

In [ ]:
import numpy as np

np.round(response.predictions, 2)

In [ ]:
endpoint.undeploy_all()  # undeploy all models from the endpoint
endpoint.delete()

## Running Batch Prediction Jobs on Vertex AI

batch_path = Path("my_mnist_batch")
batch_path.mkdir(exist_ok=True)
with open(batch_path / "my_mnist_batch.jsonl", "w") as jsonl_file:
    for image in X_test[:100].tolist():
        jsonl_file.write(json.dumps(image))
        jsonl_file.write("\n")

upload_directory(bucket, batch_path)

In [ ]:
batch_prediction_job = mnist_model.batch_predict(
    job_display_name="my_batch_prediction_job",
    machine_type="n1-standard-4",
    starting_replica_count=1,
    max_replica_count=5,
    accelerator_type="NVIDIA_TESLA_K80",
    accelerator_count=1,
    gcs_source=[f"gs://{bucket_name}/{batch_path.name}/my_mnist_batch.jsonl"],
    gcs_destination_prefix=f"gs://{bucket_name}/my_mnist_predictions/",
    sync=True  # set to False if you don't want to wait for completion
)

In [ ]:
batch_prediction_job.output_info  # extra code – shows the output directory

In [ ]:
y_probas = []
for blob in batch_prediction_job.iter_outputs():
    print(blob.name)  # extra code
    if "prediction.results" in blob.name:
        for line in blob.download_as_text().splitlines():
            y_proba = json.loads(line)["prediction"]
            y_probas.append(y_proba)

In [ ]:
y_pred = np.argmax(y_probas, axis=1)
accuracy = np.sum(y_pred == y_test[:100]) / 100

In [ ]:
accuracy

In [ ]:
mnist_model.delete()

Let's delete all the directories we created on GCS (i.e., all the blobs with these prefixes):

In [ ]:
for prefix in ["my_mnist_model/", "my_mnist_batch/", "my_mnist_predictions/"]:
    blobs = bucket.list_blobs(prefix=prefix)
    for blob in blobs:
        blob.delete()

#bucket.delete()  # uncomment and run if you want to delete the bucket itself
batch_prediction_job.delete()

# Deploying a Model to a Mobile or Embedded Device

In [ ]:
converter = tf.lite.TFLiteConverter.from_saved_model(str(model_path))
tflite_model = converter.convert()
with open("my_converted_savedmodel.tflite", "wb") as f:
    f.write(tflite_model)

In [ ]:
# extra code – shows how to convert a Keras model
converter = tf.lite.TFLiteConverter.from_keras_model(model)

In [ ]:
converter.optimizations = [tf.lite.Optimize.DEFAULT]

In [ ]:
tflite_model = converter.convert()
with open("my_converted_keras_model.tflite", "wb") as f:
    f.write(tflite_model)

# Running a Model in a Web Page

Code examples for this section are hosted on glitch.com, a website that lets you create Web apps for free.

* https://homl.info/tfjscode: a simple TFJS Web app that loads a pretrained model and classifies an image.
* https://homl.info/tfjswpa: the same Web app setup as a WPA. Try opening this link on various platforms, including mobile devices.
** https://homl.info/wpacode: this WPA's source code.
* https://tensorflow.org/js: The TFJS library.
** https://www.tensorflow.org/js/demos: some fun demos.

# Using GPUs to Speed Up Computations

Let's check that TensorFlow can see the GPU:

In [26]:
physical_gpus = tf.config.list_physical_devices("GPU")
physical_gpus

[]

If you want your TensorFlow script to use only GPUs \#0 and \#1 (based on PCI order), then you can set the environment variables `CUDA_DEVICE_ORDER=PCI_BUS_ID` and `CUDA_VISIBLE_DEVICES=0,1` before starting your script, or in the script itself before using TensorFlow.

## Managing the GPU RAM

To limit the amount of RAM to 2GB per GPU:

In [27]:
#for gpu in physical_gpus:
#    tf.config.set_logical_device_configuration(
#        gpu,
#        [tf.config.LogicalDeviceConfiguration(memory_limit=2048)]
#    )

To make TensorFlow grab memory as it needs it (only releasing it when the process shuts down):

In [29]:
#for gpu in physical_gpus:
#    tf.config.experimental.set_memory_growth(gpu, True)

Equivalently, you can set the `TF_FORCE_GPU_ALLOW_GROWTH` environment variable to `true` before using TensorFlow.

To split a physical GPU into two logical GPUs:

In [30]:
#tf.config.set_logical_device_configuration(
#    physical_gpus[0],
#    [tf.config.LogicalDeviceConfiguration(memory_limit=2048),
#     tf.config.LogicalDeviceConfiguration(memory_limit=2048)]
#)

In [31]:
logical_gpus = tf.config.list_logical_devices("GPU")
logical_gpus

[]

## Placing Operations and Variables on Devices

To log every variable and operation placement (this must be run just after importing TensorFlow):

In [32]:
#tf.get_logger().setLevel("DEBUG")  # log level is INFO by default
#tf.debugging.set_log_device_placement(True)

In [33]:
a = tf.Variable([1., 2., 3.])  # float32 variable goes to the GPU
a.device

'/job:localhost/replica:0/task:0/device:CPU:0'

In [34]:
b = tf.Variable([1, 2, 3])  # int32 variable goes to the CPU
b.device

'/job:localhost/replica:0/task:0/device:CPU:0'

You can place variables and operations manually on the desired device using a `tf.device()` context:

In [35]:
with tf.device("/cpu:0"):
    c = tf.Variable([1., 2., 3.])

c.device

'/job:localhost/replica:0/task:0/device:CPU:0'

If you specify a device that does not exist, or for which there is no kernel, TensorFlow will silently fallback to the default placement:

In [36]:
# extra code

with tf.device("/gpu:1234"):
    d = tf.Variable([1., 2., 3.])

d.device

'/job:localhost/replica:0/task:0/device:CPU:0'

If you want TensorFlow to throw an exception when you try to use a device that does not exist, instead of falling back to the default device:

In [37]:
tf.config.set_soft_device_placement(False)

# extra code
try:
    with tf.device("/gpu:1000"):
        d = tf.Variable([1., 2., 3.])
except tf.errors.InvalidArgumentError as ex:
    print(ex)

tf.config.set_soft_device_placement(True)  # extra code – back to soft placement

Could not satisfy device specification '/job:localhost/replica:0/task:0/device:GPU:1000'. enable_soft_placement=0. Supported device types [CPU]. All available devices [/job:localhost/replica:0/task:0/device:CPU:0].


## Parallel Execution Across Multiple Devices

If you want to set the number of inter-op or intra-op threads (this may be useful if you want to avoid saturating the CPU, or if you want to make TensorFlow single-threaded, to run a perfectly reproducible test case):

In [38]:
#tf.config.threading.set_inter_op_parallelism_threads(10)
#tf.config.threading.set_intra_op_parallelism_threads(10)

# Training Models Across Multiple Devices

## Training at Scale Using the Distribution Strategies API

In [39]:
# extra code – creates a CNN model for MNIST using Keras
def create_model():
    return tf.keras.Sequential([
        tf.keras.layers.Reshape([28, 28, 1], input_shape=[28, 28],
                                dtype=tf.uint8),
        tf.keras.layers.Rescaling(scale=1 / 255),
        tf.keras.layers.Conv2D(filters=64, kernel_size=7, activation="relu",
                               padding="same"),
        tf.keras.layers.MaxPooling2D(pool_size=2),
        tf.keras.layers.Conv2D(filters=128, kernel_size=3, activation="relu",
                               padding="same"), 
        tf.keras.layers.Conv2D(filters=128, kernel_size=3, activation="relu",
                               padding="same"),
        tf.keras.layers.MaxPooling2D(pool_size=2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(units=64, activation="relu"),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(units=10, activation="softmax"),
    ])

In [40]:
tf.random.set_seed(42)

strategy = tf.distribute.MirroredStrategy()

with strategy.scope():
    model = create_model()  # create a Keras model normally
    model.compile(loss="sparse_categorical_crossentropy",
                  optimizer=tf.keras.optimizers.SGD(learning_rate=1e-2),
                  metrics=["accuracy"])  # compile the model normally

batch_size = 100  # preferably divisible by the number of replicas
model.fit(X_train, y_train, epochs=10,
          validation_data=(X_valid, y_valid), batch_size=batch_size)

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)


INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)


Epoch 1/10


2023-05-21 22:37:03.383502: W tensorflow/core/framework/dataset.cc:769] Input of GeneratorDatasetOp::Dataset will not be optimized because the dataset does not implement the AsGraphDefInternal() method needed to apply optimizations.


550/550 [==============================] - ETA: 0s - loss: 1.3171 - accuracy: 0.5818

...

550/550 [==============================] - 384s 699ms/step - loss: 0.1427 - accuracy: 0.9585 - val_loss: 0.0706 - val_accuracy: 0.9784
Epoch 9/10
550/550 [==============================] - ETA: 0s - loss: 0.1316 - accuracy: 0.9623

2023-05-21 23:31:11.407982: W tensorflow/core/framework/dataset.cc:769] Input of GeneratorDatasetOp::Dataset will not be optimized because the dataset does not implement the AsGraphDefInternal() method needed to apply optimizations.


550/550 [==============================] - 386s 701ms/step - loss: 0.1316 - accuracy: 0.9623 - val_loss: 0.0648 - val_accuracy: 0.9828
Epoch 10/10
550/550 [==============================] - ETA: 0s - loss: 0.1255 - accuracy: 0.9642

2023-05-21 23:37:36.184574: W tensorflow/core/framework/dataset.cc:769] Input of GeneratorDatasetOp::Dataset will not be optimized because the dataset does not implement the AsGraphDefInternal() method needed to apply optimizations.


550/550 [==============================] - 387s 704ms/step - loss: 0.1255 - accuracy: 0.9642 - val_loss: 0.0630 - val_accuracy: 0.9820


In [41]:
type(model.weights[0])

tensorflow.python.distribute.values.MirroredVariable

In [42]:
model.predict(X_new).round(2)  # extra code – the batch is split across all replicas

1/1 [==============================] - 3s 3s/step


array([[0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.]], dtype=float32)

In [43]:
# extra code – shows that saving a model does not preserve its distribution
#              strategy
model.save("my_mirrored_model", save_format="tf")
model = tf.keras.models.load_model("my_mirrored_model")
type(model.weights[0])

INFO:tensorflow:Assets written to: my_mirrored_model/assets


INFO:tensorflow:Assets written to: my_mirrored_model/assets


tensorflow.python.ops.resource_variable_ops.ResourceVariable

In [44]:
with strategy.scope():
    model = tf.keras.models.load_model("my_mirrored_model")

In [45]:
type(model.weights[0])

tensorflow.python.distribute.values.MirroredVariable

If you want to specify the list of GPUs to use:

In [46]:
strategy = tf.distribute.MirroredStrategy(devices=["/gpu:0", "/gpu:1"])

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')


INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')


If you want to change the default all-reduce algorithm:

In [47]:
strategy = tf.distribute.MirroredStrategy(
    cross_device_ops=tf.distribute.HierarchicalCopyAllReduce())

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)


INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)


If you want to use the `CentralStorageStrategy`:

In [48]:
strategy = tf.distribute.experimental.CentralStorageStrategy()

INFO:tensorflow:ParameterServerStrategy (CentralStorageStrategy if you are using a single machine) with compute_devices = ['/job:localhost/replica:0/task:0/device:CPU:0'], variable_device = '/job:localhost/replica:0/task:0/device:CPU:0'


INFO:tensorflow:ParameterServerStrategy (CentralStorageStrategy if you are using a single machine) with compute_devices = ['/job:localhost/replica:0/task:0/device:CPU:0'], variable_device = '/job:localhost/replica:0/task:0/device:CPU:0'


In [49]:
# To train on a TPU in Google Colab:
#if "google.colab" in sys.modules and "COLAB_TPU_ADDR" in os.environ:
#  tpu_address = "grpc://" + os.environ["COLAB_TPU_ADDR"]
#else:
#  tpu_address = ""
#resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu_address)
#tf.config.experimental_connect_to_cluster(resolver)
#tf.tpu.experimental.initialize_tpu_system(resolver)
#strategy = tf.distribute.experimental.TPUStrategy(resolver)

## Training a Model on a TensorFlow Cluster

A TensorFlow cluster is a group of TensorFlow processes running in parallel, usually on different machines, and talking to each other to complete some work, for example training or executing a neural network. Each TF process in the cluster is called a "task" (or a "TF server"). It has an IP address, a port, and a type (also called its role or its job). The type can be `"worker"`, `"chief"`, `"ps"` (parameter server) or `"evaluator"`:
* Each **worker** performs computations, usually on a machine with one or more GPUs.
* The **chief** performs computations as well, but it also handles extra work such as writing TensorBoard logs or saving checkpoints. There is a single chief in a cluster. If it is not defined, then it is worker #0.
* A **parameter server** (ps) only keeps track of variable values, it is usually on a CPU-only machine.
* The **evaluator** obviously takes care of evaluation. There is usually a single evaluator in a cluster.

The set of tasks that share the same type is often called a "job". For example, the "worker" job is the set of all workers.

To start a TensorFlow cluster, you must first define it. This means specifying all the tasks (IP address, TCP port, and type). For example, the following cluster specification defines a cluster with 3 tasks (2 workers and 1 parameter server). It's a dictionary with one key per job, and the values are lists of task addresses:

In [50]:
cluster_spec = {
    "worker": [
        "machine-a.example.com:2222",     # /job:worker/task:0
        "machine-b.example.com:2222"      # /job:worker/task:1
    ],
    "ps": ["machine-a.example.com:2221"]  # /job:ps/task:0
}

Every task in the cluster may communicate with every other task in the server, so make sure to configure your firewall to authorize all communications between these machines on these ports (it's usually simpler if you use the same port on every machine).

When a task is started, it needs to be told which one it is: its type and index (the task index is also called the task id). A common way to specify everything at once (both the cluster spec and the current task's type and id) is to set the `TF_CONFIG` environment variable before starting the program. It must be a JSON-encoded dictionary containing a cluster specification (under the `"cluster"` key), and the type and index of the task to start (under the `"task"` key). For example, the following `TF_CONFIG` environment variable defines the same cluster as above, with 2 workers and 1 parameter server, and specifies that the task to start is worker \#0:

In [51]:
os.environ["TF_CONFIG"] = json.dumps({
    "cluster": cluster_spec,
    "task": {
        "type": "worker",
        "index": 0
    }})

Some platforms (e.g., Google Vertex AI) automatically set this environment variable for you.

TensorFlow's `TFConfigClusterResolver` class reads the cluster configuration from this environment variable:

In [52]:
resolver = tf.distribute.cluster_resolver.TFConfigClusterResolver()
resolver.cluster_spec()

ClusterSpec({'ps': ['machine-a.example.com:2221'], 'worker': ['machine-a.example.com:2222', 'machine-b.example.com:2222']})

In [53]:
resolver.task_type

'worker'

In [54]:
resolver.task_id

0

Now let's run a simpler cluster with just two worker tasks, both running on the local machine. We will use the `MultiWorkerMirroredStrategy` to train a model across these two tasks.

The first step is to write the training code. As this code will be used to run both workers, each in its own process, we write this code to a separate Python file, `my_mnist_multiworker_task.py`. The code is relatively straightforward, but there are a couple important things to note:
* We create the `MultiWorkerMirroredStrategy` before doing anything else with TensorFlow.
* Only one of the workers will take care of logging to TensorBoard. As mentioned earlier, this worker is called the *chief*. When it is not defined explicitly, then by convention it is worker #0.

Now let's run a simpler cluster with just two worker tasks, both running on the local machine. We will use the `MultiWorkerMirroredStrategy` to train a model across these two tasks.

The first step is to write the training code. As this code will be used to run both workers, each in its own process, we write this code to a separate Python file, `my_mnist_multiworker_task.py`. The code is relatively straightforward, but there are a couple important things to note:
* We create the `MultiWorkerMirroredStrategy` before doing anything else with TensorFlow.
* Only one of the workers will take care of logging to TensorBoard. As mentioned earlier, this worker is called the *chief*. When it is not defined explicitly, then by convention it is worker #0.

In [55]:
%%writefile my_mnist_multiworker_task.py

import tempfile
import tensorflow as tf

strategy = tf.distribute.MultiWorkerMirroredStrategy()  # at the start!
resolver = tf.distribute.cluster_resolver.TFConfigClusterResolver()
print(f"Starting task {resolver.task_type} #{resolver.task_id}")

# extra code – Load and split the MNIST dataset
mnist = tf.keras.datasets.mnist.load_data()
(X_train_full, y_train_full), (X_test, y_test) = mnist
X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

with strategy.scope():
    model = tf.keras.Sequential([
        tf.keras.layers.Reshape([28, 28, 1], input_shape=[28, 28],
                                dtype=tf.uint8),
        tf.keras.layers.Rescaling(scale=1 / 255),
        tf.keras.layers.Conv2D(filters=64, kernel_size=7, activation="relu",
                               padding="same", input_shape=[28, 28, 1]),
        tf.keras.layers.MaxPooling2D(pool_size=2),
        tf.keras.layers.Conv2D(filters=128, kernel_size=3, activation="relu",
                               padding="same"), 
        tf.keras.layers.Conv2D(filters=128, kernel_size=3, activation="relu",
                               padding="same"),
        tf.keras.layers.MaxPooling2D(pool_size=2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(units=64, activation="relu"),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(units=10, activation="softmax"),
    ])
    model.compile(loss="sparse_categorical_crossentropy",
                  optimizer=tf.keras.optimizers.SGD(learning_rate=1e-2),
                  metrics=["accuracy"])

model.fit(X_train, y_train, validation_data=(X_valid, y_valid), epochs=10)

if resolver.task_id == 0:  # the chief saves the model to the right location
    model.save("my_mnist_multiworker_model", save_format="tf")
else:
    tmpdir = tempfile.mkdtemp()  # other workers save to a temporary directory
    model.save(tmpdir, save_format="tf")
    tf.io.gfile.rmtree(tmpdir)  # and we can delete this directory at the end!

Writing my_mnist_multiworker_task.py


In a real world application, there would typically be a single worker per machine, but in this example we're running both workers on the same machine, so they will both try to use all the available GPU RAM (if this machine has a GPU), and this will likely lead to an Out-Of-Memory (OOM) error. To avoid this, we could use the `CUDA_VISIBLE_DEVICES` environment variable to assign a different GPU to each worker. Alternatively, we can simply disable GPU support, by setting `CUDA_VISIBLE_DEVICES` to an empty string.

We are now ready to start both workers, each in its own process. Notice that we change the task index:

In [56]:
%%bash --bg

export CUDA_VISIBLE_DEVICES=''
export TF_CONFIG='{"cluster": {"worker": ["127.0.0.1:9901", "127.0.0.1:9902"]},
                   "task": {"type": "worker", "index": 0}}'
python my_mnist_multiworker_task.py > my_worker_0.log 2>&1

**Note**: if you get warnings about `AutoShardPolicy`, you can safely ignore them. See [TF issue #42146](https://github.com/tensorflow/tensorflow/issues/42146) for more details.

That's it! Our TensorFlow cluster is now running, but we can't see it in this notebook because it's running in separate processes (but you can see the progress in `my_worker_*.log`).

Since the chief (worker #0) is writing to TensorBoard, we use TensorBoard to view the training progress. Run the following cell, then click on the settings button (i.e., the gear icon) in the TensorBoard interface and check the "Reload data" box to make TensorBoard automatically refresh every 30s. Once the first epoch of training is finished (which may take a few minutes), and once TensorBoard refreshes, the SCALARS tab will appear. Click on this tab to view the progress of the model's training and validation accuracy.

In [58]:
%load_ext tensorboard
%tensorboard --logdir=./my_mnist_multiworker_logs --port=6006

ERROR: Timed out waiting for TensorBoard to start. It may still be running as pid 35373.

In [ ]:
# strategy = tf.distribute.MultiWorkerMirroredStrategy(
#     communication_options=tf.distribute.experimental.CommunicationOptions(
#         implementation=tf.distribute.experimental.CollectiveCommunication.NCCL))

## Running Large Training Jobs on Vertex AI

Let's copy the training script, but add `import os` and change the save path to be the GCS path that the `AIP_MODEL_DIR` environment variable will point to:

In [59]:
%%writefile my_vertex_ai_training_task.py

import os
from pathlib import Path
import tempfile
import tensorflow as tf

strategy = tf.distribute.MultiWorkerMirroredStrategy()  # at the start!
resolver = tf.distribute.cluster_resolver.TFConfigClusterResolver()

if resolver.task_type == "chief":
    model_dir = os.getenv("AIP_MODEL_DIR")  # paths provided by Vertex AI
    tensorboard_log_dir = os.getenv("AIP_TENSORBOARD_LOG_DIR")
    checkpoint_dir = os.getenv("AIP_CHECKPOINT_DIR")
else:
    tmp_dir = Path(tempfile.mkdtemp())  # other workers use a temporary dirs
    model_dir = tmp_dir / "model"
    tensorboard_log_dir = tmp_dir / "logs"
    checkpoint_dir = tmp_dir / "ckpt"

callbacks = [tf.keras.callbacks.TensorBoard(tensorboard_log_dir),
             tf.keras.callbacks.ModelCheckpoint(checkpoint_dir)]

# extra code – Load and prepare the MNIST dataset
mnist = tf.keras.datasets.mnist.load_data()
(X_train_full, y_train_full), (X_test, y_test) = mnist
X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

# extra code – build and compile the Keras model using the distribution strategy
with strategy.scope():
    model = tf.keras.Sequential([
        tf.keras.layers.Reshape([28, 28, 1], input_shape=[28, 28],
                                dtype=tf.uint8),
        tf.keras.layers.Lambda(lambda X: X / 255),
        tf.keras.layers.Conv2D(filters=64, kernel_size=7, activation="relu",
                               padding="same", input_shape=[28, 28, 1]),
        tf.keras.layers.MaxPooling2D(pool_size=2),
        tf.keras.layers.Conv2D(filters=128, kernel_size=3, activation="relu",
                               padding="same"), 
        tf.keras.layers.Conv2D(filters=128, kernel_size=3, activation="relu",
                               padding="same"),
        tf.keras.layers.MaxPooling2D(pool_size=2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(units=64, activation="relu"),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(units=10, activation="softmax"),
    ])
    model.compile(loss="sparse_categorical_crossentropy",
                  optimizer=tf.keras.optimizers.SGD(learning_rate=1e-2),
                  metrics=["accuracy"])

model.fit(X_train, y_train, validation_data=(X_valid, y_valid), epochs=10,
          callbacks=callbacks)
model.save(model_dir, save_format="tf")

Writing my_vertex_ai_training_task.py


In [ ]:
custom_training_job = aiplatform.CustomTrainingJob(
    display_name="my_custom_training_job",
    script_path="my_vertex_ai_training_task.py",
    container_uri="gcr.io/cloud-aiplatform/training/tf-gpu.2-4:latest",
    model_serving_container_image_uri=server_image,
    requirements=["gcsfs==2022.3.0"],  # not needed, this is just an example
    staging_bucket=f"gs://{bucket_name}/staging"
)

In [ ]:
mnist_model2 = custom_training_job.run(
    machine_type="n1-standard-4",
    replica_count=2,
    accelerator_type="NVIDIA_TESLA_K80",
    accelerator_count=2,
)

Let's clean up:

In [ ]:
mnist_model2.delete()
custom_training_job.delete()
blobs = bucket.list_blobs(prefix=f"gs://{bucket_name}/staging/")
for blob in blobs:
    blob.delete()

# Hyperparameter Tuning on Vertex AI

In [61]:
%%writefile my_vertex_ai_trial.py

import argparse

parser = argparse.ArgumentParser()
parser.add_argument("--n_hidden", type=int, default=2)
parser.add_argument("--n_neurons", type=int, default=256)
parser.add_argument("--learning_rate", type=float, default=1e-2)
parser.add_argument("--optimizer", default="adam")
args = parser.parse_args()

import tensorflow as tf

def build_model(args):
    with tf.distribute.MirroredStrategy().scope():
        model = tf.keras.Sequential()
        model.add(tf.keras.layers.Flatten(input_shape=[28, 28], dtype=tf.uint8))
        for _ in range(args.n_hidden):
            model.add(tf.keras.layers.Dense(args.n_neurons, activation="relu"))
        model.add(tf.keras.layers.Dense(10, activation="softmax"))
        opt = tf.keras.optimizers.get(args.optimizer)
        opt.learning_rate = args.learning_rate
        model.compile(loss="sparse_categorical_crossentropy", optimizer=opt,
                      metrics=["accuracy"])
        return model

# extra code – loads and splits the dataset
mnist = tf.keras.datasets.mnist.load_data()
(X_train_full, y_train_full), (X_test, y_test) = mnist
X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

# extra code – use the AIP_* environment variable and create the callbacks
import os
model_dir = os.getenv("AIP_MODEL_DIR")
tensorboard_log_dir = os.getenv("AIP_TENSORBOARD_LOG_DIR")
checkpoint_dir = os.getenv("AIP_CHECKPOINT_DIR")
trial_id = os.getenv("CLOUD_ML_TRIAL_ID")
tensorboard_cb = tf.keras.callbacks.TensorBoard(tensorboard_log_dir)
early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=5)
callbacks = [tensorboard_cb, early_stopping_cb]

model = build_model(args)
history = model.fit(X_train, y_train, validation_data=(X_valid, y_valid),
                    epochs=10, callbacks=callbacks)
model.save(model_dir, save_format="tf")  # extra code

import hypertune

hypertune = hypertune.HyperTune()
hypertune.report_hyperparameter_tuning_metric(
    hyperparameter_metric_tag="accuracy",  # name of the reported metric
    metric_value=max(history.history["val_accuracy"]),  # max accuracy value
    global_step=model.optimizer.iterations.numpy(),
)

Writing my_vertex_ai_trial.py


In [ ]:
trial_job = aiplatform.CustomJob.from_local_script(
    display_name="my_search_trial_job",
    script_path="my_vertex_ai_trial.py",  # path to your training script
    container_uri="gcr.io/cloud-aiplatform/training/tf-gpu.2-4:latest",
    staging_bucket=f"gs://{bucket_name}/staging",
    accelerator_type="NVIDIA_TESLA_K80",
    accelerator_count=2,  # in this example, each trial will have 2 GPUs
)

In [ ]:
from google.cloud.aiplatform import hyperparameter_tuning as hpt

hp_job = aiplatform.HyperparameterTuningJob(
    display_name="my_hp_search_job",
    custom_job=trial_job,
    metric_spec={"accuracy": "maximize"},
    parameter_spec={
        "learning_rate": hpt.DoubleParameterSpec(min=1e-3, max=10, scale="log"),
        "n_neurons": hpt.IntegerParameterSpec(min=1, max=300, scale="linear"),
        "n_hidden": hpt.IntegerParameterSpec(min=1, max=10, scale="linear"),
        "optimizer": hpt.CategoricalParameterSpec(["sgd", "adam"]),
    },
    max_trial_count=100,
    parallel_trial_count=20,
)
hp_job.run()

In [ ]:
def get_final_metric(trial, metric_id):
    for metric in trial.final_measurement.metrics:
        if metric.metric_id == metric_id:
            return metric.value

trials = hp_job.trials
trial_accuracies = [get_final_metric(trial, "accuracy") for trial in trials]
best_trial = trials[np.argmax(trial_accuracies)]

In [ ]:
max(trial_accuracies)

In [ ]:
best_trial.id

In [ ]:
best_trial.parameters

# Extra Material – Distributed Keras Tuner on Vertex AI

Instead of using Vertex AI's hyperparameter tuning service, you can use [Keras Tuner](https://keras.io/keras_tuner/) (introduced in Chapter 10) and run it on Vertex AI VMs. Keras Tuner provides a simple way to scale hyperparameter search by distributing it across multiple machines: it only requires setting three environment variables on each machine, then running your regular Keras Tuner code on each machine. You can use the exact same script on all machines. One of the machines acts as the chief, and the others act as workers. Each worker asks the chief which hyperparameter values to try—it acts as the oracle—then the worker trains the model using these hyperparameter values, and finally it reports the model's performance back to the chief, which can then decide which hyperparameter values the worker should try next.

The three environment variables you need to set on each machine are:

* `KERASTUNER_TUNER_ID`: equal to `"chief"` on the chief machine, or a unique identifier on each worker machine, such as `"worker0"`, `"worker1"`, etc.
* `KERASTUNER_ORACLE_IP`: the IP address or hostname of the chief machine. The chief itself should generally use `"0.0.0.0"` to listen on every IP address on the machine.
* `KERASTUNER_ORACLE_PORT`: the TCP port that the chief will be listening on.

You can use distributed Keras Tuner on any set of machines. If you want to run it on Vertex AI machines, then you can spawn a regular training job, and just modify the training script to set the three environment variables properly before using Keras Tuner.

For example, the script below starts by parsing the `TF_CONFIG` environment variable, which will be automatically set by Vertex AI, just like earlier. It finds the address of the task of type `"chief"`, and it extracts the IP address or hostname, and the TCP port. It then defines the tuner ID as the task type followed by the task index, for example `"worker0"`. If the tuner ID is `"chief0"`, it changes it to `"chief"`, and it sets the IP to `"0.0.0.0"`: this will make it listen on all IPv4 address on its machine. Then it defines the environment variables for Keras Tuner. Next, the script creates a tuner, just like in Chapter 10, the it runs the search, and finally it saves the best model to the location given by Vertex AI:

In [62]:
%%writefile my_keras_tuner_search.py

import json
import os

tf_config = json.loads(os.environ["TF_CONFIG"])

chief_ip, chief_port = tf_config["cluster"]["chief"][0].rsplit(":", 1)
tuner_id = f'{tf_config["task"]["type"]}{tf_config["task"]["index"]}'
if tuner_id == "chief0":
    tuner_id = "chief"
    chief_ip = "0.0.0.0"
    # extra code – since the chief doesn't work much, you can optimize compute
    # resources by running a worker on the same machine. To do this, you can
    # just make the chief start another process, after tweaking the TF_CONFIG
    # environment variable to set the task type to "worker" and the task index
    # to a unique value. Uncomment the next few lines to give this a try:
    # import subprocess
    # import sys
    # tf_config["task"]["type"] = "workerX"  # the worker on the chief's machine
    # os.environ["TF_CONFIG"] = json.dumps(tf_config)
    # subprocess.Popen([sys.executable] + sys.argv,
    #                  stdout=sys.stdout, stderr=sys.stderr)

os.environ["KERASTUNER_TUNER_ID"] = tuner_id
os.environ["KERASTUNER_ORACLE_IP"] = chief_ip
os.environ["KERASTUNER_ORACLE_PORT"] = chief_port

from pathlib import Path
import keras_tuner as kt
import tensorflow as tf

gcs_path = "/gcs/my_bucket/my_hp_search"  # replace with your bucket's name

def build_model(hp):
    n_hidden = hp.Int("n_hidden", min_value=0, max_value=8, default=2)
    n_neurons = hp.Int("n_neurons", min_value=16, max_value=256)
    learning_rate = hp.Float("learning_rate", min_value=1e-4, max_value=1e-2,
                             sampling="log")
    optimizer = hp.Choice("optimizer", values=["sgd", "adam"])
    if optimizer == "sgd":
        optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate)
    else:
        optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Flatten(input_shape=[28, 28], dtype=tf.uint8))
    for _ in range(n_hidden):
        model.add(tf.keras.layers.Dense(n_neurons, activation="relu"))
    model.add(tf.keras.layers.Dense(10, activation="softmax"))
    model.compile(loss="sparse_categorical_crossentropy",
                  optimizer=optimizer,
                  metrics=["accuracy"])
    return model

hyperband_tuner = kt.Hyperband(
    build_model, objective="val_accuracy", seed=42,
    max_epochs=10, factor=3, hyperband_iterations=2,
    distribution_strategy=tf.distribute.MirroredStrategy(),
    directory=gcs_path, project_name="mnist")

# extra code – Load and split the MNIST dataset
mnist = tf.keras.datasets.mnist.load_data()
(X_train_full, y_train_full), (X_test, y_test) = mnist
X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

tensorboard_log_dir = os.environ["AIP_TENSORBOARD_LOG_DIR"] + "/" + tuner_id
tensorboard_cb = tf.keras.callbacks.TensorBoard(tensorboard_log_dir)
early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=5)
hyperband_tuner.search(X_train, y_train, epochs=10,
                       validation_data=(X_valid, y_valid),
                       callbacks=[tensorboard_cb, early_stopping_cb])

if tuner_id == "chief":
    best_hp = hyperband_tuner.get_best_hyperparameters()[0]
    best_model = hyperband_tuner.hypermodel.build(best_hp)
    best_model.save(os.getenv("AIP_MODEL_DIR"), save_format="tf")

Writing my_keras_tuner_search.py


Note that Vertex AI automatically mounts the `/gcs` directory to GCS, using the open source [GCS Fuse adapter](https://cloud.google.com/storage/docs/gcs-fuse). This gives us a shared directory across the workers and the chief, which is required by Keras Tuner. Also note that we set the distribution strategy to a `MirroredStrategy`. This will allow each worker to use all the GPUs on its machine, if there's more than one.


Replace `/gcs/my_bucket/` with <code>/gcs/<i>{bucket_name}</i>/</code>:

In [ ]:
with open("my_keras_tuner_search.py") as f:
    script = f.read()

with open("my_keras_tuner_search.py", "w") as f:
    f.write(script.replace("/gcs/my_bucket/", f"/gcs/{bucket_name}/"))

Now all we need to do is to start a custom training job based on this script, exactly like in the previous section. Don't forget to add `keras-tuner` to the list of `requirements`:

In [ ]:
hp_search_job = aiplatform.CustomTrainingJob(
    display_name="my_hp_search_job",
    script_path="my_keras_tuner_search.py",
    container_uri="gcr.io/cloud-aiplatform/training/tf-gpu.2-4:latest",
    model_serving_container_image_uri=server_image,
    requirements=["keras-tuner~=1.1.2"],
    staging_bucket=f"gs://{bucket_name}/staging",
)

In [ ]:
mnist_model3 = hp_search_job.run(
    machine_type="n1-standard-4",
    replica_count=3,
    accelerator_type="NVIDIA_TESLA_K80",
    accelerator_count=2,
)

And we have a model!

In [ ]:
mnist_model3.delete()
hp_search_job.delete()
blobs = bucket.list_blobs(prefix=f"gs://{bucket_name}/staging/")
for blob in blobs:
    blob.delete()